# Queries for the search application

In [34]:
import pandas as pd
import json
import pymongo
import io
import pprint
import itertools

In [2]:
#Connect to the mongoDB database
client = pymongo.MongoClient()
db = client["TWT_DB"]
tweets = db["tweets"]

In [13]:
#Document(row) sample from the mongoDB database 
myquery = {}
mydoc = list(tweets.find(myquery))
mydoc[0]

{'_id': ObjectId('5ea66a14fb8f5ff7fd471280'),
 'id_str': '1254639872641576960',
 'user_name': 'Xcntrik',
 'user_id': '123628682',
 'content': 'Many U.S. businesses unlikely to seek government aid: NABE survey https://t.co/9bPT4YQN5I https://t.co/0B3ekpqymK',
 'created_at': '2020-04-27 05:13:50',
 'location': 'Southeast Texas',
 'hashtags': [],
 'mentions': ['1652541'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '1652541',
 'media': ['https://t.co/0B3ekpqymK'],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter for Android',
 'retweet_count': 7}

## Search by word

In [22]:
#Find number of tweets with word
myquery = {"content":{"$regex":"covid19","$options" :'i'}}
tweets.count_documents(myquery)

413

In [23]:
#Tweets
twts = list(tweets.find(myquery))
twts[0]

{'_id': ObjectId('5ea66a14fb8f5ff7fd4712a9'),
 'id_str': '1254636770051928071',
 'user_name': 'Pamela Moore #WWG1WGA #DIGITALSOLDIERS',
 'user_id': '1188695886781698050',
 'content': 'Met virtually with our terrific @AmChamPH Board on how U.S. firms are supporting the Philippine economy, workforce, and #COVID19 response. We’re all in this together, and great to \u200bsee U.S. firms contributing in so many ways. #FriendsPartnersAllies #goodbUSinessPH',
 'created_at': '2020-04-27 05:01:30',
 'location': 'Broken Arrow, OK',
 'hashtags': [],
 'mentions': ['441540997', '4745918652'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '441540997',
 'media': [],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter for Android',
 'retweet_count': 118}

## Search by exact phrase

In [6]:
#Find number of tweets with #covid19 (Exact string put the specified string between the ^(Starts with) and $(Ends with))
myquery = {"hashtags":{"$regex":"^covid19$","$options" :'i'}}
tweets.count_documents(myquery)

222

In [7]:
#Tweets
twts = list(tweets.find(myquery))
twts[0]

{'_id': ObjectId('5ea66a16fb8f5ff7fd471300'),
 'id_str': '1254628238032203776',
 'user_name': 'Toshitaka Kage',
 'user_id': '181232582',
 'content': 'Fauci says #COVID19 testing needs to be doubled before U.S. reopens economy https://t.co/B576qDECyl https://t.co/A0RRjKEw8m',
 'created_at': '2020-04-27 04:27:36',
 'location': '',
 'hashtags': ['COVID19'],
 'mentions': ['487118986'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '487118986',
 'media': ['https://t.co/A0RRjKEw8m'],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter Web App',
 'retweet_count': 27}

## Search by a hashtag

In [20]:
#Search using a hashtag.
myquery = {"content":{"$regex":".#covid19.","$options" :'i'}}
twts = list(tweets.find(myquery))

In [21]:
twts[0]

{'_id': ObjectId('5ea66a14fb8f5ff7fd4712a9'),
 'id_str': '1254636770051928071',
 'user_name': 'Pamela Moore #WWG1WGA #DIGITALSOLDIERS',
 'user_id': '1188695886781698050',
 'content': 'Met virtually with our terrific @AmChamPH Board on how U.S. firms are supporting the Philippine economy, workforce, and #COVID19 response. We’re all in this together, and great to \u200bsee U.S. firms contributing in so many ways. #FriendsPartnersAllies #goodbUSinessPH',
 'created_at': '2020-04-27 05:01:30',
 'location': 'Broken Arrow, OK',
 'hashtags': [],
 'mentions': ['441540997', '4745918652'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '441540997',
 'media': [],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter for Android',
 'retweet_count': 118}

## Search by time range

In [24]:
# Search by Time range
twts = db.tweets.find({"created_at":{ "$gte": "2020-04-27 05:13:10", "$lt": "2020-04-27 05:13:50" }})
twts = list(twts)
twts[0]

{'_id': ObjectId('5ea66a14fb8f5ff7fd471281'),
 'id_str': '1254639834473230336',
 'user_name': 'Dave Talltree, son of Wymo of Mokuleia',
 'user_id': '1098317762626088961',
 'content': 'Many U.S. businesses unlikely to seek government aid: NABE survey https://t.co/9bPT4YQN5I https://t.co/0B3ekpqymK',
 'created_at': '2020-04-27 05:13:41',
 'location': 'Keiki o ka ʻĀina ',
 'hashtags': [],
 'mentions': ['1652541'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '1652541',
 'media': ['https://t.co/0B3ekpqymK'],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter Web App',
 'retweet_count': 7}

In [25]:
len(twts)

2

## Misc

In [15]:
#???????
myquery = {"location":"New York, NY"}

tweets.count_documents(myquery)

104

In [16]:
#Types of tweets
count_retweet = tweets.count_documents({"retweet":True})
count_original = tweets.count_documents({"retweet":False})
"Number of original tweet:" + str(count_original) + " and number of retweets:" + str(count_retweet)

'Number of original tweet:3306 and number of retweets:12123'

In [38]:
hashtag_count = {}
for tweet in tweets.find():
    ht = tweet["hashtags"]
    for hash in ht:
        hash = hash.lower()
        if hash not in hashtag_count:
            hashtag_count[hash] = 1
        else:
            hashtag_count[hash] += 1

hc = sorted(hashtag_count, key=hashtag_count.get, reverse=True)  # Sorting the top 10.
count = 0

for z in hc:
    if count == 10:
        break
    jsonx = {"hashtag": z, "count": str(hashtag_count[z])}
    count += 1
# {k: v for k, v in sorted(hashtag_count.items(), key=lambda item: item[1])}

In [ ]:
twts = db.tweets.runcommand( { "$text": { "$search": "\"U.S.\" \"#covid19""} } )
twts = list(twts)
twts

## Experimenting with text index

In [ ]:
#Connect to the mongoDB database
client = pymongo.MongoClient()
db = client["TWT_DB"]
tweets = db["test"]

In [ ]:
#Create a test index
db.tweets.create_index([("content",pymongo.TEXT)])

In [27]:
twts = db.tweets.find({"$text": {"$search": "\"#covid19\"" } } )
twts = list(twts)
twts[0]

{'_id': ObjectId('5ea66afbfb8f5ff7fd474756'),
 'id_str': '1253752842361483265',
 'user_name': 'Dr. Marcell Vollmer #SocialDistancing #StayHome',
 'user_id': '99674560',
 'content': 'The Great #Lockdown of the economy has been completely unprecedented, both in terms of the speed of the shutdown and its impact on jobs.\n\n #coronavirus #COVID19 #health #COVIDー19 #worklife #motivation #FutureofWork #Leadership #WorkLifeBalance #USA\n\n https://t.co/yoEHiyJcxc',
 'created_at': '2020-04-24 18:29:05',
 'location': 'Munich, Bavaria',
 'hashtags': ['Lockdown',
  'coronavirus',
  'COVID19',
  'health',
  'COVIDー19',
  'worklife',
  'motivation',
  'FutureofWork',
  'Leadership',
  'WorkLifeBalance',
  'USA'],
 'mentions': [],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': -1,
 'media': [],
 'retweet': False,
 'FavCount': 6,
 'source': 'Twitter for iPad',
 'retweet_count': 4}

In [28]:
len(twts)

396

In [29]:
twts = db.tweets.find({"$text": {"$search": "U.S."} } )
twts = list(twts)
twts[0]

{'_id': ObjectId('5ea66aecfb8f5ff7fd47442a'),
 'id_str': '1253789808247349248',
 'user_name': 'Bud Babbit',
 'user_id': '1251491717850808320',
 'content': '@IngrahamAngle New York is also responsible for 8% of the U.S. GDP.  It’s economy is the same size as Canada’s.  If NY was a country it’s total economy would sit 11th on the list....repeat, of countries.   Not shocking that it’s population created such a covid problem.',
 'created_at': '2020-04-24 20:55:59',
 'location': '',
 'hashtags': [],
 'mentions': ['50769180'],
 'in_reply_to_user_id': 50769180,
 'in_reply_to_status_id': 1253787025909723139,
 'retweetedFrom': -1,
 'media': [],
 'retweet': False,
 'FavCount': 4,
 'source': 'Twitter for iPhone',
 'retweet_count': 0}